In [46]:
import wandb
import numpy as np

In [47]:
api = wandb.Api()
entity, project = "tychovdo", "sparse"  
runs = api.runs(entity + "/" + project) 

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains output keys/values for 
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
         if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)
    
data = list(zip(summary_list, config_list, name_list))

In [48]:
def get_runs(model, kron, dataset='cifar10'):
    runs = []
    for summary, config, name in data:
        is_kron = config['n_epochs_burnin'] < 100
        if (config['model'] == model) and \
            config['dataset'] == dataset and \
            (is_kron == kron):
            runs.append((dict(summary), dict(config), list(name)))
    return runs

runs = get_runs('f_rpp', True)
print('runs:', len(runs))
run = runs[0]

runs: 2


In [49]:
def print_table1(model, dataset):
    runs = get_runs(model, True, dataset=dataset)
    run = runs[0]
    
    summary, config, name = run
    
    for layer in range(8):
        if model == 'd_rpp':
            lin_prec = summary[f'hyperparams/prior_prec_conv_net.{layer}.linear.weight']
            conv_prec = summary[f'hyperparams/prior_prec_conv_net.{layer}.conv.weight']
            
            lin_weight = summary[f'params/conv_net.{layer}.linear.weight']
            conv_weight = summary[f'params/conv_net.{layer}.conv.weight']
            
            lin_neff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.linear.weight']
            conv_neff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.conv.weight']
            
            lin_reff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.linear.weight']
            conv_reff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.conv.weight']
            total_reff = lin_reff + conv_reff
            lin_reff = lin_reff / total_reff
            conv_reff = conv_reff / total_reff
            
            if lin_reff > conv_reff:
                layer_type = 'FC'
            else:
                layer_type = 'CONV'
            
            print(f"{layer} & "
                  f"{lin_prec:8.8f} & {conv_prec:8.8f} & "
                  f"{lin_weight:8.8f} & {conv_weight:8.8f} & "
                  f"{lin_neff:8.8f} & {conv_neff:8.8f} & "
                  f"{lin_reff:8.8f} & {conv_reff:8.8f} & {layer_type} \\\\")
            
        elif model == 'f_rpp':
            pass
        elif model == 'mixer1':
            pass
        

In [50]:
model, dataset = 'd_rpp', 'cifar10'
print_table1(model, dataset)

0 & 3588348.50000000 & 0.07513507 & 0.00000000 & 10.74257360 & 0.00000002 & 0.80731778 & 0.00000003 & 0.99999997 & CONV \\
1 & 2750749.50000000 & 1.26366782 & 0.00000000 & 0.55490241 & 0.00000007 & 0.70107951 & 0.00000010 & 0.99999990 & CONV \\
2 & 5898884.50000000 & 14.50690365 & 0.00000000 & 0.04562404 & 0.00000014 & 0.66181451 & 0.00000021 & 0.99999979 & CONV \\
3 & 22690960.00000000 & 230.75096130 & 0.00000000 & 0.00216724 & 0.00000089 & 0.50004517 & 0.00000178 & 0.99999822 & CONV \\
4 & 29633508.00000000 & 5438.10449219 & 0.00000000 & 0.00007521 & 0.00000772 & 0.40905216 & 0.00001888 & 0.99998112 & CONV \\
5 & 35631228.00000000 & 3797.48779297 & 0.00000000 & 0.00003772 & 0.00000379 & 0.14326709 & 0.00002643 & 0.99997357 & CONV \\
6 & 635.57513428 & 31321916.00000000 & 0.00001135 & 0.00000000 & 0.00721232 & 0.00000522 & 0.99927627 & 0.00072373 & FC \\
7 & 35582056.00000000 & 1.75766361 & 0.00000000 & 0.00259477 & 0.00000009 & 0.00456014 & 0.00001874 & 0.99998126 & CONV \\


In [51]:
def print_table2(model, dataset):
    runs = get_runs(model, True, dataset=dataset)
    #print(len(runs))
    
    run = runs[0]
    
    summary, config, name = run
    
    lin_list, conv_list, gconv_list = [], [], []
    
    for layer in range(8):
        if model == 'd_rpp':
            pass
        elif model == 'f_rpp':
            pass
        elif model == 'mixer1':
            linu_reff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.linear.lin_U.weight']
            linv_reff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.linear.lin_V.weight']
            conv_reff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.conv.weight']
            gconv_reff = summary[f'hyperparams/norm_effect_dim_conv_net.{layer}.gconv.conv.weight']
            
            lin_reff = (linu_reff + linv_reff) / 2
            total_reff = lin_reff + conv_reff + gconv_reff
            
            lin_reff = lin_reff / total_reff
            conv_reff = conv_reff / total_reff
            gconv_reff = gconv_reff / total_reff
            
            if lin_reff > conv_reff > gconv_reff:
                layer_type = 'FC'
            elif conv_reff > lin_reff > gconv_reff:
                layer_type = 'CONV'
            else:
                layer_type = 'GCONV'
                
            lin_list.append(lin_reff)
            conv_list.append(conv_reff)
            gconv_list.append(gconv_reff)
            
            if False:
                print(f"{layer} & "\
                      f"{linu_reff:8.8f} & " \
                      f"{linv_reff:8.8f} & " \
                      f"{conv_reff:8.8f} & " \
                      f"{gconv_reff:8.8f} & {layer_type} \\")
    lin_mean = np.mean(lin_list)
    conv_mean = np.mean(conv_list)
    gconv_mean = np.mean(gconv_list)
    lin_min = np.min(lin_list)
    conv_min = np.min(conv_list)
    gconv_min = np.min(gconv_list)
    lin_max = np.max(lin_list)
    conv_max = np.max(conv_list)
    gconv_max = np.max(gconv_list)
       
    if False:
        print(f"MEAN: & "\
              f"{lin_mean:.2f} & " \
              f"{conv_mean:.2f} & " \
              f"{gconv_mean:.2f} & {layer_type} \\")   
        print(f"MIN: & "\
          f"{lin_min:.2f} & " \
          f"{conv_min:.2f} & " \
          f"{gconv_min:.2f} & {layer_type} \\")   
        print(f"MAX: & "\
          f"{lin_max:.2f} & " \
          f"{conv_max:.2f} & " \
          f"{gconv_max:.2f} & {layer_type} \\")   
    
    AA = '\\textcolor{gray}{'
    BB = '}'
    print(f""\
      f"{int(np.round(lin_mean*100))} & {AA}({int(np.round(lin_min*100))}-{int(np.round(lin_max*100))}){BB} & " \
      f"{int(np.round(conv_mean*100))} & {AA}({int(np.round(conv_min*100))}-{int(np.round(conv_max*100))}){BB} & " \
      f"{int(np.round(gconv_mean*100))} & {AA}({int(np.round(gconv_min*100))}-{int(np.round(gconv_max*100))}){BB} \\\\")       

In [52]:
model, dataset = 'mixer1', 'mnist'
print_table2(model, dataset)
model, dataset = 'mixer1', 'translated_mnist'
print_table2(model, dataset)
model, dataset = 'mixer1', 'mnist_r180'
print_table2(model, dataset)

model, dataset = 'mixer1', 'cifar10'
print_table2(model, dataset)
model, dataset = 'mixer1', 'cifar10_r180'
print_table2(model, dataset)

10 & \textcolor{gray}{(0-46)} & 15 & \textcolor{gray}{(0-98)} & 75 & \textcolor{gray}{(1-100)} \\
0 & \textcolor{gray}{(0-0)} & 23 & \textcolor{gray}{(0-99)} & 77 & \textcolor{gray}{(1-100)} \\
8 & \textcolor{gray}{(0-20)} & 8 & \textcolor{gray}{(0-47)} & 83 & \textcolor{gray}{(47-100)} \\
0 & \textcolor{gray}{(0-1)} & 44 & \textcolor{gray}{(0-99)} & 56 & \textcolor{gray}{(0-100)} \\
5 & \textcolor{gray}{(0-28)} & 14 & \textcolor{gray}{(0-41)} & 81 & \textcolor{gray}{(58-99)} \\
